In [1]:
from bokeh.plotting import figure
from bokeh.io import output_notebook, output_file, show
from bokeh.models import Range1d
from bokeh.layouts import column
from timoshenko_bending_buckling import *
import numpy as np
from scipy.interpolate import CubicSpline

#output_notebook()
output_file("timoshenko_bcb.html")

T10RGB = {
    "blue": "rgba(87, 120, 164, 1)",
    "orange": "rgba(228, 148, 68, 1)",
    "red": "rgba(209, 97, 93, 1)",
    "teal": "rgba(133, 182, 178, 1)",
    "green": "rgba(106, 159, 88, 1)",
    "yellow": "rgba(231, 202, 96, 1)",
    "purple": "rgba(168, 124, 159, 1)",
    "pink": "rgba(241, 162, 169, 1)",
    "brown": "rgba(150, 118, 98, 1)",
    "grey": "rgba(184, 176, 172, 1)"
}

T10_keys = [*T10RGB.keys()]

def solveForK(a_b,alpha):
    detC = cMatrix(a_b,1, alpha).det()
    # use sympy to solve system of equations det(C) = 0, returning [3x1] vector of solutions
    k_vals = sym.solve(detC)
    # we only care about the minimum solution (that's > 0)
    k_min = minGreaterZero(k_vals)
    return k_min

In [2]:
a_b = np.linspace(0.1,5,30, dtype=np.float32)
alpha = np.linspace(0,2,10, dtype=np.float32)

In [3]:
# generate array of buckling coefficients for every alpha
k_dict = {}
for ind, alpha_i in enumerate(alpha):
    print("solving k vals for alpha = {:.2f}...".format(alpha_i))
    k_vec_i = np.vectorize(lambda x: solveForK(x, alpha_i))(a_b)
    k_vec_i = k_vec_i.astype(np.float32)    # cast results to dtype that Bokeh supports
    key_i = "a{:.3f}".format(alpha_i)
    legend_i = "alpha = {:.2f}".format(alpha_i)
    ind_min = np.argmin(k_vec_i)        # index of minimum value
    k_min = k_vec_i[ind_min]            # min value
    
    # create ab and k vectors that asymptote to min value
    ab_asym = np.append(a_b[:ind_min+1],a_b[-1])
    k_asym = np.append(k_vec_i[:ind_min+1],k_min)
    
    # pack everything up and add to dict
    k_dict[key_i]={
        "k_vec":k_vec_i,
        "label":legend_i,
        "kmin":k_min,
        "ind_min":ind_min,
        "ab_asym": ab_asym,
        "k_asym": k_asym
    }
    

solving k vals for alpha = 0.00...
solving k vals for alpha = 0.22...
solving k vals for alpha = 0.44...
solving k vals for alpha = 0.67...
solving k vals for alpha = 0.89...
solving k vals for alpha = 1.11...
solving k vals for alpha = 1.33...
solving k vals for alpha = 1.56...
solving k vals for alpha = 1.78...
solving k vals for alpha = 2.00...


In [7]:
#fig1.line(x=a_b,y=k_vec)
#fig1.multi_line(np.array(multi_x),np.array(multi_y))#,legend_label=legend_labels)
# generate figure
TOOLTIPS = [
    ("(x,y)", "($x, $y)")
]

fig1 = figure(title="Timoshenko Bending-compression buckling curves",
              x_axis_label="a/b ratio",
              y_axis_label="Buckling coefficient, k",
              width=800,
              tools="pan,wheel_zoom,box_zoom,reset,save,hover",
              tooltips=TOOLTIPS,
              toolbar_location = "right")

fig2 = figure(title="Timoshenko Bending-compression buckling curves (asymptotic to min value)",
              x_axis_label="a/b ratio",
              y_axis_label="Buckling coefficient, k",
              width=800,
              tools="pan,wheel_zoom,box_zoom,reset,save,hover",
              tooltips=TOOLTIPS,
              toolbar_location = "right")

# loop through entries in K dict and add lines to figure
for ind, key in enumerate(k_dict):
    k_obj = k_dict[key]
    # smooth k curve using cubic spline
    k_spl = CubicSpline(a_b,k_obj["k_vec"])
    fig1.line(x = a_b,
              y = k_spl(a_b),
              legend_label = k_obj["label"],
              line_color = T10RGB[T10_keys[ind % 10]])
    
    fig2.line(x = k_obj["ab_asym"],
              y = k_obj["k_asym"],
              legend_label = k_obj["label"],
              line_color = T10RGB[T10_keys[ind % 10]])

# set other figure properties
fig1.y_range = Range1d(0,50)
fig1.add_layout(fig1.legend[0], 'right')
fig1.legend.click_policy="mute"
fig2.y_range = Range1d(0,50)
fig2.add_layout(fig2.legend[0], 'right')
fig2.legend.click_policy="mute"
# show(fig1)
# show(fig2)
show(column(fig1, fig2))